In [11]:
import numpy as np
import random

In [14]:

batch_size = 6
block_size = 12
max_iters = 200
eval_interval = 20
learning_rate = 1e-3
eval_iters = 5
n_embd = 32
n_head = 2
n_layer = 2
dropout = 0.0


text = """
Злая луна светит в небе
Ветер шепчет в тени деревьев
Тихо падают листья на землю
Ночь окутывает город туманом
Звезды мерцают далеко вдали
Сны приходят в сердце уставшем
Тишина правит в объятиях ночи
Магия живёт в каждом мгновении
"""
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = np.array(encode(text))
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    max_pos = max(1, len(data) - block_size)
    ix = np.random.randint(0, max_pos, size=(batch_size,))
    x = np.stack([data[i:i + block_size] for i in ix])
    y = np.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / e_x.sum(axis=-1, keepdims=True)

def cross_entropy(logits, targets):
    probs = softmax(logits)
    loss = -np.log(probs[np.arange(len(targets)), targets]).mean()
    return loss

class Head:
    def __init__(self, head_size):
        self.key = np.random.randn(n_embd, head_size) * 0.01
        self.query = np.random.randn(n_embd, head_size) * 0.01
        self.value = np.random.randn(n_embd, head_size) * 0.01
        self.tril = np.tril(np.ones((block_size, block_size)))

    def __call__(self, x):
        B, T, C = x.shape
        k = x @ self.key
        q = x @ self.query


        wei = q @ k.transpose(0, 2, 1) * C ** -0.5  # (B,T,T)
        wei = np.where(self.tril[:T, :T] == 0, -np.inf, wei)
        wei = softmax(wei)

        v = x @ self.value
        out = wei @ v
        return out

class MultiHeadAttention:
    def __init__(self, num_heads, head_size):
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = np.random.randn(num_heads * head_size, n_embd) * 0.01

    def __call__(self, x):
        out = np.concatenate([h(x) for h in self.heads], axis=-1)
        out = out @ self.proj
        return out

class FeedForward:
    def __init__(self, n_embd):
        self.net = [
            Linear(n_embd, 4 * n_embd),
            ReLU(),
            Linear(4 * n_embd, n_embd)
        ]

    def __call__(self, x):
        for layer in self.net:
            x = layer(x)
        return x

class Linear:
    def __init__(self, in_features, out_features):
        self.weight = np.random.randn(in_features, out_features) * 0.01
        self.bias = np.zeros(out_features)

    def __call__(self, x):
        return x @ self.weight + self.bias

class ReLU:
    def __call__(self, x):
        return np.maximum(0, x)

class LayerNorm:
    def __init__(self, dim):
        self.gamma = np.ones(dim)
        self.beta = np.zeros(dim)

    def __call__(self, x):
        mean = x.mean(axis=-1, keepdims=True)
        std = x.std(axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + 1e-5) + self.beta

class Block:
    def __init__(self, n_embd, n_head):
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = LayerNorm(n_embd)
        self.ln2 = LayerNorm(n_embd)

    def __call__(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class AdamW:
    def __init__(self, params, lr=1e-3):
        self.params = params
        self.lr = lr

    def step(self, grads):
        for p, g in zip(self.params, grads):
            p -= self.lr * g

class GPT:
    def __init__(self):
        self.token_embedding = np.random.randn(vocab_size, n_embd) * 0.01
        self.position_embedding = np.random.randn(block_size, n_embd) * 0.01
        self.blocks = [Block(n_embd, n_head) for _ in range(n_layer)]
        self.ln_f = LayerNorm(n_embd)
        self.lm_head = np.random.randn(n_embd, vocab_size) * 0.01

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding[idx]
        pos_emb = self.position_embedding[np.arange(T)]
        x = tok_emb + pos_emb

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = x @ self.lm_head

        loss = None
        if targets is not None:
            loss = cross_entropy(logits.reshape(-1, logits.shape[-1]), targets.reshape(-1))

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self.forward(idx_cond)
            logits = logits[:, -1, :]
            probs = softmax(logits)
            idx_next = np.array([np.random.choice(vocab_size, p=p) for p in probs])
            idx = np.column_stack((idx, idx_next))
        return idx


model = GPT()
params = [model.token_embedding, model.position_embedding, model.lm_head]
for block in model.blocks:
    params.extend([block.sa.proj] + [h.key for h in block.sa.heads] +
                  [h.query for h in block.sa.heads] + [h.value for h in block.sa.heads])
    params.extend([layer.weight for layer in block.ffwd.net if isinstance(layer, Linear)])

optimizer = AdamW(params, lr=learning_rate)

print(f"Размер словаря: {vocab_size}")
print(f"Длина обучающих данных: {len(train_data)}")
print(f"Длина валидационных данных: {len(val_data)}")

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = {}
        for split in ['train', 'val']:
            batch_losses = []
            for _ in range(eval_iters):
                xb, yb = get_batch(split)
                _, loss = model.forward(xb, yb)
                batch_losses.append(loss)
            losses[split] = np.mean(batch_losses)
        print(f"Итерация {iter}: Обучающая потеря {losses['train']:.4f}, Валидационная потеря {losses['val']:.4f}")

    try:
        xb, yb = get_batch('train')
        _, loss = model.forward(xb, yb)


        grads = [0.01 * np.random.randn(*p.shape) for p in params]
        optimizer.step(grads)
    except Exception as e:
        print(f"Ошибка на итерации {iter}: {str(e)}")
        break


print("\nСгенерированный текст:")
context = np.array([[stoi['С']]])
generated_indices = model.generate(context, max_new_tokens=50)[0]
print(decode(generated_indices))


Размер словаря: 37
Длина обучающих данных: 184
Длина валидационных данных: 47
Итерация 0: Обучающая потеря 3.6082, Валидационная потеря 3.6223
Итерация 20: Обучающая потеря 3.6027, Валидационная потеря 3.6191
Итерация 40: Обучающая потеря 3.6072, Валидационная потеря 3.6164
Итерация 60: Обучающая потеря 3.6093, Валидационная потеря 3.6164
Итерация 80: Обучающая потеря 3.6101, Валидационная потеря 3.6224
Итерация 100: Обучающая потеря 3.6090, Валидационная потеря 3.6169
Итерация 120: Обучающая потеря 3.6072, Валидационная потеря 3.6115
Итерация 140: Обучающая потеря 3.6102, Валидационная потеря 3.6283
Итерация 160: Обучающая потеря 3.6104, Валидационная потеря 3.6144
Итерация 180: Обучающая потеря 3.6066, Валидационная потеря 3.6246

Сгенерированный текст:
СгЗлпМшТТгТсакццсзпяю
уъзМТаЗ юххязтысьтмсъгт 
л СМ
